In [29]:
import torch
import numpy as np
import utils
import models
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import os

In [30]:
def get_noise(b_size=1):
        return torch.randn([b_size, noise_size], device=device)

def generate_random_chair(condition):
    with torch.no_grad():
        noise = get_noise()
        condition_tensor = torch.tensor([[condition]], dtype=torch.float, device=device)
        fake = generator(noise, condition_tensor)
    return fake.squeeze().cpu().numpy()

def plot(voxel1,voxel2,voxel3):
    fig = plt.figure(figsize=(10, 5))

    # First subplot
    ax1 = fig.add_subplot(131, projection='3d')
    ax1.voxels(voxel1 >0.5, facecolors='red', edgecolor='k')
    ax1.set_title('Voxel 1')

    # Second subplot
    ax2 = fig.add_subplot(132, projection='3d')
    ax2.voxels(voxel2>0.5, facecolors='blue', edgecolor='k')
    ax2.set_title('Voxel 2')

    ax3 = fig.add_subplot(133, projection='3d')
    ax3.voxels(voxel3>0.5, facecolors='blue', edgecolor='k')
    ax3.set_title('Merged')

    plt.show()

def calculate_AAD(generated_models, merged_model):
    n = len(generated_models)
    total_elements = np.prod(generated_models[0].shape)
    
    sum_diff = 0
    for model in generated_models:
        sum_diff += np.sum(np.abs(model - merged_model))
    
    AAD = sum_diff / (total_elements * n)
    return AAD

def calculate_AVAR(generated_models, merged_model, threshold=0.5):
    n = len(generated_models)
    
    sum_ratio = 0
    for model in generated_models:
        binary_model = (model > threshold).astype(int)
        binary_merged = (merged_model > threshold).astype(int)
        
        agreement = np.sum(np.logical_and(binary_model, binary_merged))
        total_voxels = np.sum(binary_model)
        
        sum_ratio += agreement / total_voxels
    
    AVAR = sum_ratio / n
    return AVAR

def Preds():
    Gen1  =generate_random_chair(0)
    Gen2  =np.rot90(generate_random_chair(1), 2)
    return [Gen1,Gen2]

def evaluate_batch( batch_size):
    AADs = []
    AVARs = []
    
    for _ in range(batch_size):
        
        # Generate models for each condition
        aligned_models =  Preds()

        
        # Create merged model
        merged_model = np.mean(aligned_models, axis=0)
        
        # Calculate metrics
        AAD = calculate_AAD(aligned_models, merged_model)
        AVAR = calculate_AVAR(aligned_models, merged_model)
        
        AADs.append(AAD)
        AVARs.append(AVAR)
    
    return np.mean(AADs), np.mean(AVARs)

In [31]:
def Preds_Baseline():
    Gen1  =generate_random_chair(0)
    Gen2  =generate_random_chair(0)
    return [Gen1,Gen2]

def evaluate_batch_BaseLine( batch_size):
    AADs = []
    AVARs = []
    
    for _ in range(batch_size):
        
        # Generate models for each condition
        aligned_models =  Preds_Baseline()

        
        # Create merged model
        merged_model = np.mean(aligned_models, axis=0)
        
        # Calculate metrics
        AAD = calculate_AAD(aligned_models, merged_model)
        AVAR = calculate_AVAR(aligned_models, merged_model)
        
        AADs.append(AAD)
        AVARs.append(AVAR)
    
    return np.mean(AADs), np.mean(AVARs)

In [32]:
cube_len = 32
noise_size = 200
condition_count = 2
moo = os.listdir('Baselline_models')
for i in moo:
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        # Initialize the generator
    generator = models.Generator(noise_size=(noise_size + 1), cube_resolution=cube_len)
    generator = generator.to(device)

        # Load the trained model
    generator.load_state_dict(torch.load(f'Proposed_models/{i}', map_location=device))
    generator.eval()

    # Evaluate baseline and proposed method
    proposed_AAD, proposed_AVAR = evaluate_batch( batch_size=128)
    print(f"Model: {i}")
    print(f"Proposed_models: AAD = {proposed_AAD:.4f}, AVAR = {proposed_AVAR:.4f}")

    generator = models.Generator(noise_size=(noise_size + 1), cube_resolution=cube_len)
    generator = generator.to(device)

        # Load the trained model
    generator.load_state_dict(torch.load(f'Baselline_models/{i}', map_location=device))
    generator.eval()

    baseline_AAD, baseline_AVAR = evaluate_batch( batch_size=128)
    print(f"Baseline: AAD = {baseline_AAD:.4f}, AVAR = {baseline_AVAR:.4f}")

    print("==============================================================")

Model: BCE_L1_model_state_dict_280.pth
Proposed_models: AAD = 0.0138, AVAR = 0.6022
Baseline: AAD = 0.0151, AVAR = 0.4999
Model: BCE_L1_model_state_dict_360.pth
Proposed_models: AAD = 0.0152, AVAR = 0.6447
Baseline: AAD = 0.0098, AVAR = 0.4696
Model: BCE_L1_model_state_dict_400.pth
Proposed_models: AAD = 0.0132, AVAR = 0.5772
Baseline: AAD = 0.0111, AVAR = 0.3429
Model: CustomHybridLossBM_model_state_dict_280.pth
Proposed_models: AAD = 0.0634, AVAR = 0.2386
Baseline: AAD = 0.0113, AVAR = 0.3973
Model: CustomHybridLossBM_model_state_dict_360.pth
Proposed_models: AAD = 0.0496, AVAR = 0.2172
Baseline: AAD = 0.0107, AVAR = 0.3046
Model: CustomHybridLossBM_model_state_dict_400.pth
Proposed_models: AAD = 0.0485, AVAR = 0.2411
Baseline: AAD = 0.0097, AVAR = 0.3898
Model: CustomHybridLossMB_model_state_dict_280.pth
Proposed_models: AAD = 0.0156, AVAR = 0.5389
Baseline: AAD = 0.0109, AVAR = 0.4191
Model: CustomHybridLossMB_model_state_dict_360.pth
Proposed_models: AAD = 0.0150, AVAR = 0.5897
Ba

C:\Users\varun\AppData\Local\Temp\ipykernel_9340\521991117.py:52: RuntimeWarning: invalid value encountered in scalar divide
  sum_ratio += agreement / total_voxels


Baseline: AAD = 0.0028, AVAR = nan
Model: Triple_Loss_model_state_dict_360.pth
Proposed_models: AAD = 0.0172, AVAR = 0.5670
Baseline: AAD = 0.0050, AVAR = 0.1298
Model: Triple_Loss_model_state_dict_400.pth
Proposed_models: AAD = 0.0158, AVAR = 0.5371
Baseline: AAD = 0.0070, AVAR = 0.2614
